**Movie Recommendation System - Collaborative Denoising Auto-Encoders**

In [ ]:
# ===== Block 1: Setup Environment =====

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from scipy import sparse
import random

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Fix random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


Mounted at /content/drive
Using device: cuda


In [ ]:
# ===== Block 2: Data Download and Preprocessing =====

import pandas as pd
import numpy as np
from scipy import sparse
import os

# Step 1: Download MovieLens 25M dataset
!wget -q --show-progress https://files.grouplens.org/datasets/movielens/ml-25m.zip
!unzip -q ml-25m.zip

# Step 2: Load ratings.csv
ratings = pd.read_csv('ml-25m/ratings.csv')

# Step 3: Filter users with at least 5 ratings
user_counts = ratings['userId'].value_counts()
filtered_users = user_counts[user_counts >= 5].index
ratings = ratings[ratings['userId'].isin(filtered_users)]

# Step 4: Map userId and movieId to continuous indices
user2idx = {user_id: idx for idx, user_id in enumerate(ratings['userId'].unique())}
item2idx = {item_id: idx for idx, item_id in enumerate(ratings['movieId'].unique())}

ratings['userId'] = ratings['userId'].map(user2idx)
ratings['movieId'] = ratings['movieId'].map(item2idx)

num_users = ratings['userId'].nunique()
num_items = ratings['movieId'].nunique()

print(f"Total users: {num_users}, Total items: {num_items}")

# Step 5: Create Interaction Matrix
interaction_matrix = sparse.lil_matrix((num_users, num_items))
for row in ratings.itertuples():
    interaction_matrix[row.userId, row.movieId] = 1

# Step 6: Save Interaction Matrix
os.makedirs('/content/drive/MyDrive/CDAE_Final_Model', exist_ok=True)
sparse.save_npz('/content/drive/MyDrive/CDAE_Final_Model/interaction_matrix.npz', interaction_matrix.tocsr())

print("Interaction matrix saved successfully at /content/drive/MyDrive/CDAE_Final_Model/interaction_matrix.npz")


ml-25m.zip.1        100%[===================>] 249.84M  81.3MB/s    in 3.1s    
replace ml-25m/tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
Total users: 162541, Total items: 59047
Interaction matrix saved successfully at /content/drive/MyDrive/CDAE_Final_Model/interaction_matrix.npz


In [ ]:
# ===== Block 3: Model Definition (CDAE) =====

class CDAE(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=200, dropout_rate=0.5):
        super(CDAE, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim

        # User embedding
        self.user_embedding = nn.Embedding(num_users, embedding_dim)

        # Encoder
        self.encoder = nn.Linear(num_items, embedding_dim)

        # Decoder
        self.decoder = nn.Linear(embedding_dim, num_items)

        # Dropout
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, user_id):
        user_emb = self.user_embedding(user_id)
        x = self.dropout(x)
        x = self.encoder(x)
        x = x + user_emb
        x = torch.sigmoid(x)
        x = self.decoder(x)
        x = torch.sigmoid(x)
        return x


In [ ]:
# ===== Block 4: Training Setup =====

from torch.utils.data import Dataset, DataLoader

# Dataset Class
class InteractionDataset(Dataset):
    def __init__(self, interaction_matrix):
        self.interactions = interaction_matrix

    def __len__(self):
        return self.interactions.shape[0]

    def __getitem__(self, idx):
        return torch.FloatTensor(self.interactions[idx].toarray().flatten()), torch.LongTensor([idx])

# Load saved interaction matrix
from scipy import sparse
interaction_matrix = sparse.load_npz('/content/drive/MyDrive/CDAE_Final_Model/interaction_matrix.npz')

# Create train-validation split
full_dataset = InteractionDataset(interaction_matrix)

train_size = int(0.9 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

# Initialize model, optimizer, scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CDAE(num_users=num_users, num_items=num_items).to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

# Loss Function
criterion = nn.BCELoss()


In [ ]:
!pip install tqdm


In [ ]:
# ===== Block 5: Training Loop with Evaluation and Progress Bar =====

import os
import numpy as np
from tqdm import tqdm  # <-- Progress bar import

# ===== Setup paths =====
checkpoint_dir = '/content/drive/MyDrive/CDAE_Checkpoints'
final_model_dir = '/content/drive/MyDrive/CDAE_Final_Model'
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(final_model_dir, exist_ok=True)

# ===== Define evaluation metrics =====
def recall_at_k(predictions, ground_truth, k=10):
    top_k = predictions.topk(k, dim=1)[1]
    correct = (ground_truth.gather(1, top_k) > 0).float()
    recall = correct.sum(1) / torch.clamp(ground_truth.sum(1), min=1.0)
    return recall.mean().item()

def ndcg_at_k(predictions, ground_truth, k=10):
    top_k = predictions.topk(k, dim=1)[1]
    gains = (ground_truth.gather(1, top_k)).float()
    discounts = torch.log2(torch.arange(2, k + 2, device=ground_truth.device).float())
    dcg = (gains / discounts).sum(1)
    ideal_gains = torch.sort(ground_truth, descending=True)[0][:, :k]
    ideal_dcg = (ideal_gains / discounts).sum(1)
    ndcg = (dcg / torch.clamp(ideal_dcg, min=1e-10)).mean().item()
    return ndcg

# ===== Training Settings =====
EPOCHS = 130
start_epoch = 0

# ===== Training Loop =====
for epoch in range(start_epoch, EPOCHS):
    model.train()
    epoch_loss = 0.0

    # Progress bar for training batches
    loop = tqdm(train_loader, leave=True, desc=f"Epoch {epoch+1}/{EPOCHS}")

    for user_vectors, user_ids in loop:
        user_vectors = user_vectors.to(device)
        user_ids = user_ids.squeeze().to(device)

        optimizer.zero_grad()
        outputs = model(user_vectors, user_ids)

        loss = criterion(outputs, user_vectors)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        loop.set_postfix(loss=loss.item())

    scheduler.step()

    # ===== Evaluation after each epoch =====
    model.eval()
    recall_scores = []
    ndcg_scores = []

    with torch.no_grad():
        for user_vectors, user_ids in validation_loader:
            user_vectors = user_vectors.to(device)
            user_ids = user_ids.squeeze().to(device)

            outputs = model(user_vectors, user_ids)

            recall_scores.append(recall_at_k(outputs, user_vectors, k=10))
            ndcg_scores.append(ndcg_at_k(outputs, user_vectors, k=10))

    mean_recall = np.mean(recall_scores)
    mean_ndcg = np.mean(ndcg_scores)

    print(f"\nEpoch {epoch+1}/{EPOCHS} Completed | Avg Loss: {epoch_loss/len(train_loader):.4f} | Recall@10: {mean_recall:.4f} | NDCG@10: {mean_ndcg:.4f}\n")

    # ===== Save checkpoint every 15 epochs =====
    if (epoch + 1) % 15 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'cdae_checkpoint_epoch_{epoch+1}.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict()
        }, checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")

# ===== Save Final Model after Training =====
final_model_path = os.path.join(final_model_dir, 'cdae_final_model.pth')
torch.save({
    'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict()
}, final_model_path)
print(f"Final model saved successfully at {final_model_path}")


Epoch 1/130: 100%|██████████| 286/286 [01:32<00:00,  3.08it/s, loss=0.011]



Epoch 1/130 Completed | Avg Loss: 0.0164 | Recall@10: 0.0507 | NDCG@10: 0.4484



Epoch 2/130: 100%|██████████| 286/286 [01:23<00:00,  3.43it/s, loss=0.0127]



Epoch 2/130 Completed | Avg Loss: 0.0120 | Recall@10: 0.0504 | NDCG@10: 0.4458



Epoch 3/130: 100%|██████████| 286/286 [01:22<00:00,  3.45it/s, loss=0.0125]


...


Epoch 128/130 Completed | Avg Loss: 0.0060 | Recall@10: 0.1176 | NDCG@10: 0.7844



Epoch 129/130: 100%|██████████| 286/286 [01:24<00:00,  3.40it/s, loss=0.00573]



Epoch 129/130 Completed | Avg Loss: 0.0060 | Recall@10: 0.1175 | NDCG@10: 0.7843



Epoch 130/130: 100%|██████████| 286/286 [01:23<00:00,  3.40it/s, loss=0.00612]



Epoch 130/130 Completed | Avg Loss: 0.0060 | Recall@10: 0.1175 | NDCG@10: 0.7842

Final model saved successfully at /content/drive/MyDrive/CDAE_Final_Model/cdae_final_model.pth


In [ ]:
# ===== Block 6: Final Full Evaluation (Recall/NDCG @ 10, 100, 500) =====

import os
import numpy as np

# ===== Paths =====
evaluation_result_path = '/content/drive/MyDrive/CDAE_Final_Model/final_evaluation_results.txt'

# ===== Load Final Model (Optional, if you need) =====
# (Assume model already loaded, trained and moved to eval mode)

# ===== Define evaluation metrics (already defined if above) =====
# recall_at_k and ndcg_at_k functions already exist

# ===== Evaluate =====
model.eval()
recall_scores_10 = []
ndcg_scores_10 = []
recall_scores_100 = []
ndcg_scores_100 = []
recall_scores_500 = []
ndcg_scores_500 = []

with torch.no_grad():
    for user_vectors, user_ids in validation_loader:
        user_vectors = user_vectors.to(device)
        user_ids = user_ids.squeeze().to(device)

        outputs = model(user_vectors, user_ids)

        recall_scores_10.append(recall_at_k(outputs, user_vectors, k=10))
        ndcg_scores_10.append(ndcg_at_k(outputs, user_vectors, k=10))
        recall_scores_100.append(recall_at_k(outputs, user_vectors, k=100))
        ndcg_scores_100.append(ndcg_at_k(outputs, user_vectors, k=100))
        recall_scores_500.append(recall_at_k(outputs, user_vectors, k=500))
        ndcg_scores_500.append(ndcg_at_k(outputs, user_vectors, k=500))

# ===== Mean Scores =====
mean_recall_10 = np.mean(recall_scores_10)
mean_ndcg_10 = np.mean(ndcg_scores_10)
mean_recall_100 = np.mean(recall_scores_100)
mean_ndcg_100 = np.mean(ndcg_scores_100)
mean_recall_500 = np.mean(recall_scores_500)
mean_ndcg_500 = np.mean(ndcg_scores_500)

# ===== Save Results to Text File =====
with open(evaluation_result_path, 'w') as f:
    f.write("===== Final Evaluation Results =====\n")
    f.write(f"Recall@10: {mean_recall_10:.4f}\n")
    f.write(f"NDCG@10: {mean_ndcg_10:.4f}\n\n")
    f.write(f"Recall@100: {mean_recall_100:.4f}\n")
    f.write(f"NDCG@100: {mean_ndcg_100:.4f}\n\n")
    f.write(f"Recall@500: {mean_recall_500:.4f}\n")
    f.write(f"NDCG@500: {mean_ndcg_500:.4f}\n")

print(f"Evaluation completed! Results saved at: {evaluation_result_path}")


Evaluation completed! Results saved at: /content/drive/MyDrive/CDAE_Final_Model/final_evaluation_results.txt


Recommendation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import os


In [ ]:
# ===== Setup Paths =====
final_model_dir = '/content/drive/MyDrive/CDAE_Final_Model'
movies_csv_path = os.path.join(final_model_dir, 'movies_backup.csv')  # Updated

# ===== Load interaction matrix =====
interaction_matrix = sparse.load_npz(os.path.join(final_model_dir, 'interaction_matrix.npz'))

# ===== Load movie titles =====
movies_df = pd.read_csv(movies_csv_path)

# ===== Define Model Architecture (MATCHES TRAINING) =====
class CDAE(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=200, dropout_rate=0.5):  # Match
        super(CDAE, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim

        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.encoder = nn.Linear(num_items, embedding_dim)
        self.decoder = nn.Linear(embedding_dim, num_items)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, user_id):
        user_emb = self.user_embedding(user_id)
        x = self.dropout(x)
        x = self.encoder(x)
        x = x + user_emb
        x = torch.sigmoid(x)
        x = self.decoder(x)
        x = torch.sigmoid(x)
        return x

# ===== Setup Device =====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ===== Initialize Correct Model =====
num_users, num_items = interaction_matrix.shape
model = CDAE(num_users=num_users, num_items=num_items, embedding_dim=200, dropout_rate=0.5).to(device)

# ===== Load Saved Weights =====
model_checkpoint = torch.load(os.path.join(final_model_dir, 'cdae_final_model.pth'), map_location=device)
model.load_state_dict(model_checkpoint['model_state_dict'])
model.eval()

print("Model loaded successfully with correct architecture!")


Model loaded successfully with correct architecture!


In [ ]:
def recommend_movies(user_id, top_n=10):
    """
    Recommend top N movies for a given user id.
    """
    model.eval()
    with torch.no_grad():
        user_vector = interaction_matrix[user_id].toarray().flatten()
        input_vector = torch.FloatTensor(user_vector).unsqueeze(0).to(device)
        user_tensor = torch.LongTensor([user_id]).to(device)

        predictions = model(input_vector, user_tensor)
        scores = predictions.cpu().numpy().flatten()

        # Filter out already watched movies
        watched_items = np.where(user_vector > 0)[0]
        scores[watched_items] = -np.inf  # Set watched movies score to -inf

        top_indices = scores.argsort()[-top_n:][::-1]

        recommended_titles = []
        for idx in top_indices:
            movie_id = idx
            # Map back to movie title
            movie_title = movies_df[movies_df['movie_encoded'] == movie_id]['title'].values
            if len(movie_title) > 0:
                recommended_titles.append(movie_title[0])

        return recommended_titles


In [ ]:
# Recommend movies for a random user
user_id = 123  # Example user within [0, num_users-1]
recommended = recommend_movies(user_id, top_n=10)

print(f"\nTop 10 recommended movies for User ID {user_id}:\n")
for idx, title in enumerate(recommended, 1):
    print(f"{idx}. {title}")



Top 10 recommended movies for User ID 123:

1. Murder, My Sweet (1944)
2. Brother of Sleep (Schlafes Bruder) (1995)
3. White Balloon, The (Badkonake sefid) (1995)
4. Grace of My Heart (1996)
5. Bed of Roses (1996)
6. Die Hard (1988)
7. Outlaw, The (1943)
8. Beautiful Thing (1996)
9. Dumbo (1941)
10. Star Wars: Episode IV - A New Hope (1977)
